In [11]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams, get_associated_token_address
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [12]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [13]:
program_id = PublicKey("8Ga7yNCYJiJ6HLTZMu14iNh6una9AGZvuM4n9napx1v3")

sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)


In [14]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
avatar_mint = PublicKey("8Z7vfkpxENuAV1joZZZDLmQmkUspgJuMhkLiUbq6jcjT")
avatar_program = PublicKey("DvTvXKZR9wveQzL3pBM7QJ9YZpConSXBXRTyDsEADNqX")
governor_mint = PublicKey("Aus1568aZYNXkqhHv9j6BNx5mpbmWURkQAnpHRXuxfBf")
payer_keypair.public_key

2ZHc9QxDDaJwqNEFzpAGUrvxCWZSNnXSffHxV9hG2axp

In [15]:
# mint_account = Keypair.generate()
storage_bump = 1;

vault_pda, _ = PublicKey.find_program_address([b"Governor_Vault"], program_id)

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
(all_loans_pda, _all_loans_pda_bump) = PublicKey.find_program_address([b"All Governor Loans"], program_id);
(storage_pda, storage_pda_bump) = PublicKey.find_program_address([b"Loan_Storage", bytes(payer_keypair.public_key), storage_bump.to_bytes(4,"big")], program_id);
governor_data_pda, _ = PublicKey.find_program_address([b"governor_data_pda", bytes(governor_mint)], program_id)


mint_account_meta = AccountMeta(governor_mint, False, False)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
sales_pda_meta = AccountMeta(sales_pda, False, False)
associated_address = get_associated_token_address(payer_keypair.public_key, mint_account_meta.pubkey)
associated_account_meta = AccountMeta(associated_address, False, True)
all_loans_meta = AccountMeta(all_loans_pda, False, True)
storage_account_meta = AccountMeta(storage_pda, False, True)
storage_pda_associated = get_associated_token_address(storage_pda, mint_account_meta.pubkey)
storage_associated_meta = AccountMeta(storage_pda_associated, False, True)
vault_account_meta = AccountMeta(vault_pda, False, True)
governor_data_meta = AccountMeta(governor_data_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
associated_program_meta = AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False)


In [16]:
accounts = [
    payer_account_meta,
    # mint_account_meta,
    # sales_pda_meta,
    associated_account_meta,
    all_loans_meta,
    storage_account_meta,
    storage_associated_meta,
    vault_account_meta,
    # governor_data_meta,
    # spl_program_meta,
    # sys_program_meta,
    # rent_account_meta,
    sysvar_clock_meta,
    spl_program_meta,

    sys_program_meta,
    associated_program_meta, 
    spl_program_meta, 
    sys_program_meta, 
    
]

In [17]:
    
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([4,0,0,0,255,2,108,0,0,storage_bump])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: incorrect program id for instruction', 'data': {'accounts': None, 'err': {'InstructionError': [0, 'IncorrectProgramId']}, 'logs': ['Program 8Ga7yNCYJiJ6HLTZMu14iNh6una9AGZvuM4n9napx1v3 invoke [1]', 'Program 11111111111111111111111111111111 invoke [2]', 'Program 11111111111111111111111111111111 success', 'Program 8Ga7yNCYJiJ6HLTZMu14iNh6una9AGZvuM4n9napx1v3 consumed 29611 of 200000 compute units', 'Program 8Ga7yNCYJiJ6HLTZMu14iNh6una9AGZvuM4n9napx1v3 failed: incorrect program id for instruction'], 'unitsConsumed': 0}}

In [ ]:
time.sleep(10)

Payment = CStruct(
    "amount" / U64,
    "time" / U32
)

accountSchema = CStruct(
    "mint" / U8[32],
    "initial_amount" / U64,
    "amount_left" / U64,
    "paymeents" / Vec(Payment),
    "monthly_interest_numerator" / U32,
    "monthly_interest_denominator" / U32,
    "initial_loan_date" / U32,
    "last_pay_date" / U32,
    "is_loan_active" / Bool
)

getAccountInfo(client, storage_account_meta.pubkey).amount_left/(10**9)

0.618